# 0. Setup

In [1]:
import pandas as pd
import requests
import json
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go

import yfinance as yf

# 1. API Request

In [2]:
url = 'https://api.hkma.gov.hk/public/market-data-and-statistics/daily-monetary-statistics/daily-figures-interbank-liquidity?offset=0&pagesize=999&sortby=end_of_date&sortorder=desc'
response = requests.get(url).text
response = json.loads(response)
df = pd.DataFrame(response['result']['records'])
df['end_of_date'] = pd.to_datetime(df['end_of_date'])

In [13]:
df

,end_of_date,cu_weakside,cu_strongside,disc_win_base_rate,hibor_overnight,hibor_fixing_1m,twi,opening_balance,closing_balance,market_activities,...,forex_trans_t4,other_market_activities_t4,reversal_of_discount_window_t4,interest_payment_issuance_efbn_t4,forecast_aggregate_bal_t4,forex_trans_u,other_market_activities_u,reversal_of_discount_window_u,interest_payment_issuance_efbn_u,forecast_aggregate_bal_u
0,2025-08-14,7.85,7.75,4.75,0.32,1.04524,102.9,64156,57091,-7065,...,None,None,None,None,NaN,+0,+0,-0,-404,53311
1,2025-08-13,7.85,7.75,4.75,0.16,0.91036,103.0,64062,64156,+0,...,None,None,None,None,NaN,+0,+0,-0,-404,53311
2,2025-08-12,7.85,7.75,4.75,0.15,0.90893,103.3,64062,64062,+0,...,None,None,None,None,NaN,+0,+0,-0,-404,56687
3,2025-08-11,7.85,7.75,4.75,0.15,0.95202,103.1,64062,64062,+0,...,None,None,None,None,NaN,+0,+0,-0,-368,63752
4,2025-08-08,7.85,7.75,4.75,0.21,0.95893,103.1,72501,64062,-8439,...,None,None,None,None,NaN,+0,+0,-0,-368,63752
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,2021-08-04,7.85,7.75,0.50,0.02,0.08125,101.2,457456,457457,0,...,None,None,None,None,NaN,0,0,0,-3,457454
995,2021-08-03,7.85,7.75,0.50,0.03,0.08821,101.3,457456,457456,0,...,None,None,None,None,NaN,0,0,0,-3,457454
996,2021-08-02,7.85,7.75,0.50,0.02,0.08250,101.4,457456,457456,0,...,None,None,None,None,NaN,0,0,0,-2,457454
997,2021-07-30,7.85,7.75,0.50,0.02,0.08268,101.3,457456,457456,0,...,None,None,None,None,NaN,0,0,0,-2,457454


# 2.0 HIBOR & Liquidity

## 2.1 HIBOR (Overnight & 1-Month)

In [4]:
hibor_df = df[['end_of_date', 'hibor_overnight', 'hibor_fixing_1m']]

In [5]:
hibor_fig = px.line(hibor_df,
             x='end_of_date', y=['hibor_overnight', 'hibor_fixing_1m'],
             title='HIBOR Rates (Overnight & 1-Month)',
             labels={'end_of_date': 'Date', 'value': 'HIBOR Rate (%)'},
             )

last_date = hibor_df['end_of_date'].iloc[0]
last_overnight = hibor_df['hibor_overnight'].iloc[0]
last_1m = hibor_df['hibor_fixing_1m'].iloc[0]

hibor_fig.add_annotation(
    text=f"Date: {last_date.strftime('%Y-%m-%d')}<br>Overnight: {last_overnight}%<br>1M: {last_1m}%",
    xref="paper", yref="paper",
    x=1.19, y=0.2,  
    showarrow=False,
    bgcolor="rgba(1, 108, 2, 1)",
    borderwidth=2,
    font=dict(size=12, color="white")
)

hibor_fig.update_layout(
    xaxis_rangeslider_visible=True,
)

hibor_fig.show()

## 2.2 Aggregate Balance Open & Close

In [6]:
aggreBal_df = df[['end_of_date', 'opening_balance', 'closing_balance']].head(200)
aggreBal_df['day_change'] = aggreBal_df['closing_balance'] - aggreBal_df['opening_balance']
aggreBal_df['high'] = aggreBal_df[['opening_balance', 'closing_balance']].max(axis=1)
aggreBal_df['low'] = aggreBal_df[['opening_balance', 'closing_balance']].min(axis=1)
aggreBal_df

,end_of_date,opening_balance,closing_balance,day_change,high,low
0,2025-08-14,64156,57091,-7065,64156,57091
1,2025-08-13,64062,64156,94,64156,64062
2,2025-08-12,64062,64062,0,64062,64062
3,2025-08-11,64062,64062,0,64062,64062
4,2025-08-08,72501,64062,-8439,72501,64062
...,...,...,...,...,...,...
195,2024-10-29,44790,44790,0,44790,44790
196,2024-10-28,44790,44790,0,44790,44790
197,2024-10-25,44790,44790,0,44790,44790
198,2024-10-24,44790,44790,0,44790,44790


In [7]:
aggreBal_fig = go.Figure(data=[go.Candlestick(
    x=aggreBal_df['end_of_date'],
    open=aggreBal_df['opening_balance'],
    high=aggreBal_df['high'],
    low=aggreBal_df['low'],
    close=aggreBal_df['closing_balance'],
    name='Aggregate Balance',
    showlegend=True,
)])

last_date = aggreBal_df['end_of_date'].iloc[0]
last_open = aggreBal_df['opening_balance'].iloc[0]
last_close = aggreBal_df['closing_balance'].iloc[0]
last_day_change = aggreBal_df['day_change'].iloc[0]

aggreBal_fig.add_annotation(
    text=f"Date: {last_date.strftime('%Y-%m-%d')}<br>Open: {last_open}<br>Close: {last_close}<br>Change: {last_day_change}",
    xref="paper", yref="paper",
    x=1.2, y=0.0,
    showarrow=False,
    bgcolor="rgba(0, 78, 123, 1)",
    borderwidth=2,
    font=dict(size=12, color="white")
)

aggreBal_fig.update_layout(
    xaxis_rangeslider_visible=True,
    title="Hong Kong Aggregate Balance (Candlestick)",
    xaxis_title="Date",
    yaxis_title="Balance (HKD Million)",
)
aggreBal_fig.show()

# 3.0 FX

# 3.1 Convertibility Undertaking (CU)

In [8]:
usdhkd = yf.Ticker("HKD=X")
usdhkd_df = usdhkd.history(period ='max').reset_index()
usdhkd_df['Date'] = pd.to_datetime(usdhkd_df['Date'], format='%Y-%m-%d')
usdhkd_df = usdhkd_df[['Date', 'Close']]
usdhkd_df = usdhkd_df.rename(columns={'Date': 'end_of_date', 'Close': 'usdhkd_close'})
usdhkd_df['end_of_date'] = pd.to_datetime(usdhkd_df['end_of_date'])
usdhkd_df.set_index('end_of_date', inplace=True)
usdhkd_df.index = usdhkd_df.index.tz_localize(None)

In [9]:
cu_df = df[['end_of_date', 'cu_weakside', "cu_strongside"]]
cu_df['end_of_date'] = pd.to_datetime(cu_df['end_of_date'])
cu_df = cu_df.set_index('end_of_date').join(usdhkd_df, on='end_of_date', how='left')

/var/folders/h_/dmgktz0n27g262j_0g_v4qvm0000gn/T/ipykernel_46906/3866773713.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
cu_df = cu_df.reset_index()
cu_df.sort_values(by='end_of_date', ascending=False, inplace=True)
cu_df

,end_of_date,cu_weakside,cu_strongside,usdhkd_close
0,2025-08-14,7.85,7.75,7.83380
1,2025-08-13,7.85,7.75,7.84838
2,2025-08-12,7.85,7.75,7.84968
3,2025-08-11,7.85,7.75,7.84960
4,2025-08-08,7.85,7.75,7.84932
...,...,...,...,...
994,2021-08-04,7.85,7.75,7.77710
995,2021-08-03,7.85,7.75,7.77375
996,2021-08-02,7.85,7.75,7.77210
997,2021-07-30,7.85,7.75,7.77004


In [11]:
cu_fig = px.line(cu_df,
             x='end_of_date', y=['usdhkd_close', 'cu_weakside', 'cu_strongside'],
             title='USD/HKD and Currency Pegs',
             labels={'end_of_date': 'Date', 'value': 'Value (HKD)'},
             )


last_date = cu_df['end_of_date'].iloc[0]
last_usdhkd_close = round(cu_df['usdhkd_close'].iloc[0],4)

cu_fig.add_annotation(
    text=f"Date: {last_date.strftime('%Y-%m-%d')}<br>usdhkd: {last_usdhkd_close}",
    xref="paper", yref="paper",
    x=1.19, y=0.2,  
    showarrow=False,
    bgcolor="rgba(108, 1, 2, 1)",
    borderwidth=2,
    font=dict(size=12, color="white")
)

cu_fig.update_layout(
    xaxis_rangeslider_visible=True,
)

cu_fig.show()

# 3.2 Trade-Weighted Index, TWI

In [ ]:
hkdtwi_df = df[['end_of_date', 'twi']]

,end_of_date,twi
0,2025-08-14,102.9
1,2025-08-13,103.0
2,2025-08-12,103.3
3,2025-08-11,103.1
4,2025-08-08,103.1
...,...,...
994,2021-08-04,101.2
995,2021-08-03,101.3
996,2021-08-02,101.4
997,2021-07-30,101.3


In [29]:
hkdtwi_fig = px.line(hkdtwi_df,
             x='end_of_date', y='twi',
             title='HKD Trade-Weighted Index (TWI)',
             labels={'end_of_date': 'Date', 'value': 'Value'},
             )


last_date = hkdtwi_df['end_of_date'].iloc[0]
last_twi = hkdtwi_df['twi'].iloc[0]

hkdtwi_fig.add_annotation(
    text=f"Date: {last_date.strftime('%Y-%m-%d')}<br>TWI(HKD): {last_twi}",
    xref="paper", yref="paper",
    x=1, y=1.2,  
    showarrow=False,
    bgcolor="rgba(156, 33, 315, 1)",
    borderwidth=2,
    font=dict(size=12, color="white")
)

hkdtwi_fig.update_layout(
    xaxis_rangeslider_visible=True,
)

hkdtwi_fig.show()

# 4.0 Stock Market